In [1]:
# import important libraries
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# load the transaction dataset
df_retail = pd.read_csv('retail_transactions.csv')

In [3]:
# remove unnecessary columns
df_retail = df_retail.drop(columns=['StockCode', 'Description'])

# keep UK records only
df_retail = df_retail[df_retail['Country'] == 'United Kingdom']

# fix the data type and parse datetime
df_retail['CustomerID'] = df_retail['CustomerID'].astype(str)
df_retail['InvoiceDate'] = pd.to_datetime(df_retail['InvoiceDate']).dt.normalize()

# calculate revenue and transaction year
df_retail['Revenue'] = df_retail['UnitPrice'] * df_retail['Quantity']
df_retail['Year'] = df_retail['InvoiceDate'].dt.year

# take a look at the current dataset
df_retail.info()

<class 'pandas.core.frame.DataFrame'>
Index: 354321 entries, 0 to 397883
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    354321 non-null  int64         
 1   Quantity     354321 non-null  int64         
 2   InvoiceDate  354321 non-null  datetime64[ns]
 3   UnitPrice    354321 non-null  float64       
 4   CustomerID   354321 non-null  object        
 5   Country      354321 non-null  object        
 6   Revenue      354321 non-null  float64       
 7   Year         354321 non-null  int32         
dtypes: datetime64[ns](1), float64(2), int32(1), int64(2), object(2)
memory usage: 23.0+ MB


In [4]:
# take a look at the dataframe
df_retail.head()

,InvoiceNo,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue,Year
0,546729,12,2020-03-16,1.25,18231.0,United Kingdom,15.0,2020
1,559898,6,2020-07-13,1.25,16225.0,United Kingdom,7.5,2020
2,548648,24,2020-04-01,0.85,12949.0,United Kingdom,20.4,2020
3,540543,4,2020-01-09,2.95,14395.0,United Kingdom,11.8,2020
4,561390,10,2020-07-27,1.65,17068.0,United Kingdom,16.5,2020


In [5]:
# understand the numeric columns
print(df_retail.shape)
df_retail.describe()

(354321, 8)


,InvoiceNo,Quantity,InvoiceDate,UnitPrice,Revenue,Year
count,354321.00000,354321.000000,354321,354321.000000,354321.000000,354321.000000
mean,560672.52133,12.013795,2020-07-10 14:41:59.850644224,2.963994,20.626470,2019.932429
min,536365.00000,1.000000,2019-12-01 00:00:00,0.001000,0.001000,2019.000000
25%,549241.00000,2.000000,2020-04-07 00:00:00,1.250000,4.160000,2020.000000
50%,561900.00000,4.000000,2020-07-31 00:00:00,1.950000,10.200000,2020.000000
75%,572295.00000,12.000000,2020-10-23 00:00:00,3.750000,17.700000,2020.000000
max,581586.00000,80995.000000,2020-12-09 00:00:00,8142.750000,168469.600000,2020.000000
std,13168.90343,189.267956,NaN,17.862655,326.044012,0.251009


In [6]:
# First we would like to summarize the total amount of revenue for each invoice.

df_retail = df_retail.groupby('InvoiceNo').agg(
    {
        'Revenue':'sum',
        'CustomerID': 'first',
        'InvoiceDate': 'first',
        'Year': 'first'
    }
)
# print(df_retail.shape)
df_retail.head()
# df_retail.to_csv('3.2.retail_transactions_preprocessed.csv')

,Revenue,CustomerID,InvoiceDate,Year
InvoiceNo,,,,
536365,139.12,17850.0,2019-12-01,2019
536366,22.20,17850.0,2019-12-01,2019
536367,278.73,13047.0,2019-12-01,2019
536368,70.05,13047.0,2019-12-01,2019
536369,17.85,13047.0,2019-12-01,2019


In [7]:
df_retail.shape

(16646, 4)

In [8]:
# Narrow our focus to the transactions of 2019 only.

df_retail_19 = df_retail[df_retail['Year'].isin([2019])].copy()
df_retail_19['DaysSince'] = (dt.datetime(year=2019, month=12, day=31) - df_retail_19['InvoiceDate']).apply(lambda x: x.days)
df_retail_19.head()

,Revenue,CustomerID,InvoiceDate,Year,DaysSince
InvoiceNo,,,,,
536365,139.12,17850.0,2019-12-01,2019,30
536366,22.20,17850.0,2019-12-01,2019,30
536367,278.73,13047.0,2019-12-01,2019,30
536368,70.05,13047.0,2019-12-01,2019,30
536369,17.85,13047.0,2019-12-01,2019,30


In [9]:
# In this step we will create features for each individual customers.

operations = {
    'Revenue': 'sum',
    'DaysSince': ['max','min','nunique']
}
df_retail_19 = df_retail_19.groupby('CustomerID').agg(operations)
df_retail_19.columns = ['_'.join(col).strip() for col in df_retail_19.columns.values]
df_retail_19['AvgOrderCost'] = df_retail_19['Revenue_sum'] / df_retail_19['DaysSince_nunique']
df_retail_19


,Revenue_sum,DaysSince_max,DaysSince_min,DaysSince_nunique,AvgOrderCost
CustomerID,,,,,
12747.0,706.27,26,18,2,353.135000
12748.0,4228.13,30,8,14,302.009286
12826.0,155.00,22,22,1,155.000000
12829.0,85.75,17,17,1,85.750000
12838.0,390.79,30,30,1,390.790000
...,...,...,...,...,...
18239.0,438.10,29,29,1,438.100000
18245.0,365.73,12,12,1,365.730000
18259.0,376.30,23,23,1,376.300000


In [10]:
# Finally, we'll take the total revenue of a customer in 2020 as the label or dependent variable

revenue_2020 = df_retail[df_retail['Year'] == 2020].groupby('CustomerID')['Revenue'].sum()
revenue_2020.head()

CustomerID
12346.0    77183.60
12747.0     3489.74
12748.0    29491.60
12749.0     4090.88
12820.0      942.34
Name: Revenue, dtype: float64

In [11]:
wrangled_df = pd.concat([df_retail_19, revenue_2020], axis=1, join="inner")
wrangled_df = wrangled_df.rename(columns={
    'Revenue_sum': 'revenue_2019',
    'DaysSince_max': 'customer_relation_days',
    'DaysSince_min': 'last_purchase_days',
    'DaysSince_nunique': 'number_of_purchases',
    'AvgOrderCost': 'avg_order_cost',
    'Revenue': 'revenue_2020'})
wrangled_df.index.name = 'customer_id'

wrangled_df.to_csv('wrangled_transactions.csv')
wrangled_df.head()

,revenue_2019,customer_relation_days,last_purchase_days,number_of_purchases,avg_order_cost,revenue_2020
customer_id,,,,,,
12747.0,706.27,26,18,2,353.135000,3489.74
12748.0,4228.13,30,8,14,302.009286,29491.60
12826.0,155.00,22,22,1,155.000000,1319.72
12829.0,85.75,17,17,1,85.750000,207.25
12838.0,390.79,30,30,1,390.790000,292.34
